<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Dense_1_20_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from pgmpy.estimators import HillClimbSearch, BicScore, AICScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
import matplotlib.pyplot as plt

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (dense)

In [12]:
# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the dense Bayesian Network
dense_model = BayesianNetwork([('IR', 'EI'), ('EI', 'SP'), ('IR', 'SP')])

# Function to generate CPDs
def generate_cpds():
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_given_ir_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI given IR probabilities
    ei_df = pd.DataFrame(ei_probs, columns=['EI_given_IR_low', 'EI_given_IR_medium', 'EI_given_IR_high'])
    ei_df['EI_State'] = ['poor', 'average', 'good']

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes(data_dense, filename):
    data_dense['IR'] = data_dense['IR'].map(ir_map)
    data_dense['EI'] = data_dense['EI'].map(ei_map)
    data_dense['SP'] = data_dense['SP'].map(sp_map)
    data_dense.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the dense model
sample_sizes = range(1000, 10500, 1000)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs_reshaped = generate_cpds()

    # Define CPDs for the dense model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_dense = TabularCPD(variable='EI', variable_card=3,
                              values=ei_given_ir_probs,
                              evidence=['IR'], evidence_card=[3])
    cpd_sp_dense = TabularCPD(variable='SP', variable_card=3,
                              values=sp_probs_reshaped,
                              evidence=['IR', 'EI'], evidence_card=[3, 3])

    dense_model.add_cpds(cpd_ir, cpd_ei_dense, cpd_sp_dense)

    # Check if the model is valid
    assert dense_model.check_model()

    # Generate samples
    sampler_dense = BayesianModelSampling(dense_model)
    data_dense = sampler_dense.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities(ir_probs, ei_given_ir_probs, sp_probs_reshaped, f'probabilities_dense_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes(data_dense, f'outcomes_dense_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the dense model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the dense model!


# Hypothesis Model: 500, 1000, 1500, ..., 10000 Samples (dense) 1 hidden Layer, 20 Neurons Relu

In [13]:
# Sample sizes to loop through
sample_sizes = range(1000, 10500, 1000)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=20):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths if necessary)
    outcomes_file = f'outcomes_dense_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sample sizes complete!")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 1000
Training Data: (700, 2) (700,)
Validation Data: (150, 2) (150,)
Test Data: (150, 2) (150,)
Validation Accuracy for 1000 samples: 0.4267
Test Accuracy for 1000 samples: 0.4733
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 

Predicted Results and Probabilities for 1000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     poor     increase       0.332502     0.217107       0.450391
1   medium     poor     increase       0.168336     0.408878       0.422786
2      low     poor     increase       0.122594     0.363284       0.514122
3   medium     poor     increase       0.122594     0.363284       0.514122
4     high  average     increase       0.307112     0.297185       0.395703
5     high  average     increase       0.122594     0.363284       0.514122
6     high     good     increase       0.168336     0.408878       0.422786
7      low     poor     increase       0.122594     0.363284       0.514122
8     high    

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 2000 samples: 0.4233
Test Accuracy for 2000 samples: 0.4633
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

Predicted Results and Probabilities for 2000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     good     decrease       0.420550     0.293478       0.285971
1      low     good     decrease       0.420550     0.293478       0.285971
2      low     good       stable       0.227914     0.559879       0.212207
3     high     good     decrease       0.481437     0.302401       0.216162
4     high     poor       stable       0.320581     0.341463       0.337956
5   medium     poor     decrease       0.504647     0.347715       0.147639
6     high     good       stable       0.312025     0.392311       0.295663
7   medium     poor       stable       0.264955     0.444117       0.290928
8     high     poor       stable       0.264955     0.444117       0.290928
9   medium     poor       stable       0.26813

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Validation Accuracy for 8000 samples: 0.5150
Test Accuracy for 8000 samples: 0.5283
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 8000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     good     increase       0.343926     0.234681       0.421393
1   medium     good     decrease       0.677516     0.185248       0.137236
2   medium     poor     increase       0.357120     0.231186       0.411695
3     high     good       stable       0.185819     0.609594       0.204586
4   medium     good     decrease       0.677516     0.185248       0.137236
5      low  average     decrease       0.677516     0.185248       0.137236
6     high     poor     decrease       0.677516     0.185248       0.137236
7      low  average     increase       0.343926     0.234681       0.421393
8   medium     good     decrease       0.677516     0.185248       0.137236
9      low  average       stable       0.158113

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9000 samples: 0.4689
Test Accuracy for 9000 samples: 0.4541
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     poor       stable       0.237416     0.503705       0.258879
1   medium  average     decrease       0.480403     0.132218       0.387379
2   medium     poor       stable       0.237416     0.503705       0.258879
3      low     good       stable       0.237416     0.503705       0.258879
4   medium     good     decrease       0.480403     0.132218       0.387379
5     high  average     decrease       0.480403     0.132218       0.387379
6   medium     poor       stable       0.237416     0.503705       0.258879
7      low     poor       stable       0.237416     0.503705       0.258879
8     high     poor     increase       0.097690     0.427836       0.474474
9     high     poor       stable       0.280685

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.4807
Test Accuracy for 10000 samples: 0.4580
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     poor     increase       0.117551     0.405812       0.476637
1     high     poor     increase       0.360320     0.194611       0.445070
2   medium  average     increase       0.117551     0.405812       0.476637
3      low  average     decrease       0.510854     0.122730       0.366416
4   medium     poor       stable       0.269489     0.493754       0.236757
5      low     good       stable       0.122900     0.562428       0.314672
6      low  average     increase       0.117551     0.405812       0.476637
7      low  average     decrease       0.443761     0.189656       0.366584
8   medium     good       stable       0.122900     0.562428       0.314672
9   medium  average     increase       0.305

# K-L Divergence NN Dense Data

In [14]:
# Define the function to save K-L divergence and std dev to a file
def save_kl_divergence(sample_size, kl_div_value, std_kl_div_value, first_run=False):
    file_name = 'kl_div_NN_1_20_dense.csv'

    # If it's the first run, write the headers
    if first_run:
        with open(file_name, 'w') as f:
            f.write('Size,NN_Dense_1_20_Entropy,Std_Dev\n')  # Write headers for the CSV

    # Append the K-L divergence and std dev for this sample size to the file
    with open(file_name, 'a') as f:
        f.write(f"{sample_size},{kl_div_value:.4f},{std_kl_div_value:.4f}\n")

# Sample sizes to loop through
sample_sizes = range(1000, 10500, 1000)  # Loop from 500 to 10,000 in increments of 500

# Loop through each sample size
first_run = True  # Flag to indicate the first run for writing headers

for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size
    ground_truth_probs_file = f'probabilities_dense_{sample_size}.csv'
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Load the predictions from memory (already generated by the NN)
    outcomes_file = f'outcomes_dense_{sample_size}.csv'
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR and EI
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Assuming predictions were saved in memory from the NN run
    predictions = nn_model.predict(df[['IR_encoded', 'EI_encoded']])

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Calculate the standard deviation of the K-L divergence values
    std_kl_divergence = np.std(kl_divergences)

    # Save the K-L divergence and std dev value to a CSV file
    save_kl_divergence(sample_size, average_kl_divergence, std_kl_divergence, first_run=first_run)

    # Print confirmation
    print(f"Average K-L Divergence for {sample_size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

    # After the first run, set `first_run` to False
    first_run = False

# Once all sample sizes are processed, the K-L divergences and std devs will be saved in 'kl_div_NN_1_20_dense.csv'
print("\nK-L divergence and standard deviation calculations complete.")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Average K-L Divergence for 1000 samples: 0.1586, Std Dev: 0.2603
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 2000 samples: 0.1178, Std Dev: 0.0837
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Average K-L Divergence for 3000 samples: 0.5603, Std Dev: 0.6792
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 4000 samples: 0.1053, Std Dev: 0.1511
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 5000 samples: 0.1409, Std Dev: 0.1374
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 6000 samples: 0.5474, Std Dev: 0.2214
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 7000 samples: 0.1940, Std Dev: 0.1823
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 8000 samples: 0.1437, Std Dev: 0.1965
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 9000 samples: 0.3559, Std Dev: 0.2353
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Average K-